# Desnormalizando e analisando o Sakila

## Imports

In [15]:
import matplotlib.pyplot as plt
import pandas as pd

## Questão 1: Qual gênero de filme é mais alugado? E qual gênero de filme tem o maior preço médio de locação?

Para responder a primeira pergunta a seguinte consulta SQL foi feita:

~~~ mysql
SELECT  
	film.film_id,
	film.title,
	film.rental_duration,
	film.rental_rate,
	film_category.category_id,
	category.name
FROM film 
LEFT JOIN film_category ON film.film_id = film_category.film_id
LEFT JOIN category ON film_category.category_id = category.category_id
~~~

Isso nos retorna uma tabela com os filmes, seus títulos, por quanto tempo foram alugados, o preço de locação e seu gênero. Com isso conseguimos agrupar os filmes por gênero e analisar tanto o preço quanto a duração da locação para descobrirmos o gênero mais alugado e mais caro. Exportamos a tabela resultante da query para o formato csv e assim é possível usar a biblioteca Pandas para fazer a análise.

In [3]:
film_genres = pd.read_csv('q1.csv')
film_genres.head()

,film_id,title,rental_duration,rental_rate,category_id,name
0,1,ACADEMY DINOSAUR,6,0.99,6,Documentary
1,2,ACE GOLDFINGER,3,4.99,11,Horror
2,3,ADAPTATION HOLES,7,2.99,6,Documentary
3,4,AFFAIR PREJUDICE,5,2.99,11,Horror
4,5,AFRICAN EGG,6,2.99,8,Family


In [9]:
grouped_genres = film_genres.groupby('name').mean()[['rental_duration', 'rental_rate']]
grouped_genres

,rental_duration,rental_rate
name,,
Action,4.953125,2.646250
Animation,4.893939,2.808182
Children,5.033333,2.890000
Classics,5.070175,2.744386
Comedy,4.931034,3.162414
Documentary,4.764706,2.666471
Drama,5.080645,3.022258
Family,5.173913,2.758116
Foreign,5.109589,3.099589


In [11]:
grouped_genres.idxmax()

rental_duration    Travel
rental_rate         Games
dtype: object

Agrupando os filmes por gênero e usando a média como função de agregação para as variáveis que estamos interessados, obtemos a tabela acima. Com a função idxmax() vemos que o gênero que tem maior tempo de locação médio, ou seja, o gênero mais alugado é "Travel" e o gênero com maior preço de locação médio é "Games".

Para fins de comparação, podemos ilustrar os resultados da tabela acima em forma de gráfico.

In [ ]:
grouped_genres.plot(y = ['rental_duration', 'rental_rate'], kind = 'bar')

## Questão 2: 

## Questão 3:

## Questão 4: Apresente evidências que os dados do Sakila são, ou não reais